# Laboratorio 1 – Comparación de Bases de Datos NoSQL (MongoDB, Redis, HBase)

**Objetivo:**
Este laboratorio tiene como objetivo montar servidores de **MongoDB**, **Redis** y **HBase** en contenedores Docker, y luego interactuar con ellos desde Python para:

1. Cargar un dataset real de compras electrónicas.
2. Ejecutar consultas específicas en cada base de datos:
   - ¿Cuál es la **categoría** más vendida?
   - ¿Cuál **marca** generó más ingresos brutos?
   - ¿Qué **mes** tuvo más ventas? (en UTC)
3. Medir y comparar los tiempos de ejecución entre las tres bases de datos.

**Dataset utilizado:**
[Ecommerce Purchase History from Electronics Store](https://www.kaggle.com/datasets/mkechinov/ecommerce-purchase-history-from-electronics-store)

**Pasos principales:**
1. Preparar el entorno de Python en esta máquina virtual.
2. Descargar y preparar el dataset.
3. Insertar datos en **MongoDB**, **Redis** y **HBase**.
4. Ejecutar las consultas en cada base de datos.
5. Analizar los tiempos de respuesta.

---


In [ ]:
# Instalación de librerías necesarias
!pip3 install --user pandas pymongo redis happybase thriftpy2 kaggle


## 📥 Descarga del Dataset desde Kaggle

Para trabajar con el dataset real, utilizaremos la API de Kaggle.  
Esto requiere que tengamos un **token de acceso** que puedes obtener en tu cuenta de Kaggle:

1. Inicia sesión en [Kaggle](https://www.kaggle.com/).
2. Ve a **Account** → **Create New API Token**.
3. Esto descargará un archivo llamado `kaggle.json`.
4. Lo subiremos a esta máquina en la carpeta `~/.kaggle/`.

Este archivo contiene:
- `username`: tu usuario en Kaggle.
- `key`: clave de acceso para autenticación.


In [ ]:
import os

# Crear carpeta .kaggle si no existe
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

print("✅ Carpeta .kaggle lista. Sube tu archivo kaggle.json a esta ruta en la VM:")

# Mostrar ruta destino
print(os.path.expanduser("~/.kaggle/kaggle.json"))


## 🔓 Ajustar permisos y descargar dataset

Después de subir el archivo `kaggle.json` a la carpeta `.kaggle/` debemos ajustar los permisos para que sea seguro.  
Luego utilizaremos la librería `kaggle` para descargar el dataset directamente en esta máquina.

El dataset se guardará en la carpeta `datasets/ecommerce/`.


In [ ]:
# Ajustar permisos del token Kaggle
!chmod 600 ~/.kaggle/kaggle.json

# Descargar el dataset desde Kaggle
!kaggle datasets download -d mkechinov/ecommerce-purchase-history-from-electronics-store -p ./datasets/ecommerce --unzip

print("✅ Dataset descargado y descomprimido en ./datasets/ecommerce")


In [ ]:
import os

os.listdir("./datasets/ecommerce")


## 📂 Carga y vista previa del Dataset

El dataset descargado desde Kaggle contiene información sobre el historial de compras en una tienda de electrónica.  
El archivo descargado se llama `kz.csv` y lo cargaremos usando **Pandas** para inspeccionarlo antes de insertarlo en las bases de datos.

Pasos:
1. Leer el archivo CSV.
2. Mostrar las primeras filas para validar el contenido.
3. Revisar la estructura de las columnas.


In [ ]:
import pandas as pd

# Ruta al archivo descargado
file_path = "./datasets/ecommerce/kz.csv"

# Cargar el dataset
df = pd.read_csv(file_path)

# Vista previa de las primeras filas
df.head()


## 📊 Inspección de la estructura del Dataset

Antes de proceder a insertar los datos en las bases de datos, es importante entender su estructura y contenido.

Pasos:
1. Revisar la información general (`info()`) para conocer número de filas, columnas y tipos de datos.
2. Verificar estadísticas descriptivas (`describe()`) para entender rangos y distribución de datos numéricos.
3. Detectar posibles valores nulos o faltantes.


In [ ]:
# Información general del dataset
print("=== Información General del Dataset ===")
print(df.info())

# Estadísticas descriptivas de columnas numéricas
print("\n=== Estadísticas Descriptivas ===")
print(df.describe())

# Conteo de valores nulos por columna
print("\n=== Valores Nulos por Columna ===")
print(df.isnull().sum())


## 📥 Inserción de datos en MongoDB

Conexión y carga de datos en MongoDB usando `pymongo`.

Pasos:
1. Conectarse al servidor de MongoDB.
2. Seleccionar la base de datos (`ecommerce_db`) y la colección (`purchases`).
3. Convertir el DataFrame de Pandas a diccionarios (`to_dict('records')`).
4. Insertar los datos con `insert_many()`.

> ⚠️ **Nota:** El contenedor MongoDB debe estar corriendo y accesible en el puerto `27017`.


In [ ]:
from pymongo import MongoClient

# Parámetros de conexión
MONGO_USER = "admin"  # o "admin" si lo configuraste así
MONGO_PASS = "pass"   # tu clave actual
MONGO_HOST = "localhost"  # Si está en el mismo servidor que Jupyter
MONGO_PORT = 27017

# Conexión a MongoDB
client = MongoClient(f"mongodb://{MONGO_USER}:{MONGO_PASS}@{MONGO_HOST}:{MONGO_PORT}/?authSource=admin")
db = client["ecommerce_db"]
collection = db["purchases"]

# Conversión del DataFrame a lista de diccionarios
data_dict = df.to_dict("records")

# Inserción en MongoDB
result = collection.insert_many(data_dict)
print(f"✅ Insertados {len(result.inserted_ids)} documentos en MongoDB")
